# Introduction

This notebook illustrates the basics of how to use `plottr` -- in particular, the `inspectr` and `autoplot` tools -- to live plot data in a qcodes database.

## Basic notebook setup

In [ ]:
DBPATH = './qcodes_liveplot_demo.db'

import qcodes as qc

qc.config.core.db_location = DBPATH
qc.initialise_database()

# Launching inspectr

Next, we need to run the inspectr tool from the command line in a separate process. From within the plottr root directory, run 

``
$ python apps/inspectr.py --dbpath=./doc/examples/qcodes_liveplot_demo.db
``

We should now have two windows open; no data is yet shown if we started with a fresh .db file. 
Now, before populating the database, let's enable automatic monitoring of the dataset. To do that, enter a refresh interval (given in seconds) in the inspectr window toolbar, and enable the auto-plot option.

# Dummy experiments

Below are a few dummy qcodes experiments that should hopefully illustrate how the live plotter behaves. Run them while the inspectr is open, and monitoring is active (or not -- you can also refresh manually by pressing 'R'; this works for both inspectr and the autoplotter). 

## Qcodes imports (and other relevant stuff)

In [ ]:
import time
import numpy as np
from qcodes import load_or_create_experiment, Measurement, Parameter

## A very simple 1D sweep

This is the most basic measurement type we can imagine: sweep one independent parameter (`x`) and record data, point-by-point, as a function of that. 
Here we have two dependents, `y` and `y2`.

In plottr, you'll see a window with the the two line traces for the dependents.

In [ ]:
xvals = np.linspace(0, 10, 101)
yvals = np.sin(xvals)
y2vals = np.cos(xvals)

def simple_1d_sweep():
    for x, y, y2 in zip(xvals, yvals, y2vals):
        yield x, y, y2
        
x = Parameter('x')
y = Parameter('y')
y2 = Parameter('y2')

station = qc.Station(x, y, y2)

In [ ]:
exp = load_or_create_experiment('very_simple_1d_sweep', sample_name='no sample')

meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y, setpoints=(x,))
meas.register_parameter(y2, setpoints=(x,))
meas.write_period = 2

with meas.run() as datasaver:
    for xval, yval, y2val in simple_1d_sweep():
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (y2, y2val),
        )
        time.sleep(0.2)

## A very simple 2D sweep

In exactly the same fashion, we can also take higher-dimensional data. 
For 2D data, this means nested loops in the easiest case.

We'll now see plottr slowly rastering the data as it gets saved.

In [ ]:
# set up the dummy data
xvals = np.linspace(-5, 5, 51)
yvals = np.linspace(-5, 5, 51)
xx, yy = np.meshgrid(xvals, yvals, indexing='ij')
zz = np.cos(xx) * np.cos(yy)

def very_simple_2d_sweep():
    for i, x in enumerate(xvals):
        for j, y in enumerate(yvals):
            yield x, y, zz[i, j]

# configure the qcodes setup
x = Parameter('x')
y = Parameter('y')
z = Parameter('z')
station = qc.Station(x, y, z)
exp = load_or_create_experiment('very_simple_2d_sweep', sample_name='no sample')

# set up the measurement
meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y)
meas.register_parameter(z, setpoints=(x, y))
meas.write_period = 2

# and start recording
with meas.run() as datasaver:
    for xval, yval, zval in very_simple_2d_sweep():
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (z, zval),
        )
        time.sleep(0.2)

## A simple 2D sweep, with 1D in 'hardware'

Instead of sweeping point-by-point, it is also often the case that we get not single values, but whole arrays from a measurement call. 
This makes data acquisition much faster, and is handled in essentially the same way.
The only difference in the example below is now that the 'measurement' returns arrays for `y` and `z` (e.g., the y-dependence of z could be something that is hardware-controlled in the lab), and that both have set `paramtype='array'` in the qcodes measurement and data objects.

In [ ]:
# set up mock data
xvals = np.linspace(-5, 5, 51)
yvals = np.linspace(-5, 5, 51)
xx, yy = np.meshgrid(xvals, yvals, indexing='ij')
zz = np.cos(xx) * np.cos(yy)

def simple_2d_sweep():
    for i, x in enumerate(xvals):
        yield x, yy[i, :], zz[i, :]

# configure qcodes setup
x = Parameter('x')
y = Parameter('y')
z = Parameter('z')
station = qc.Station(x, y, z)
exp = load_or_create_experiment('simple_2d_sweep', sample_name='no sample')

# set up measurement
meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y, paramtype='array')
meas.register_parameter(z, setpoints=(x, y), paramtype='array')
meas.write_period = 2

# start measuring
with meas.run() as datasaver:
    for xval, yval, zval in simple_2d_sweep():
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (z, zval),
        )
        time.sleep(0.2)

## Complex data

Often, in particular when measuring in rf, our data is complex-valued. 
This example shows that we can plot complex data as well, and can choose between real/imaginary and magnitude/phase representation in plottr.
The data here is mocking the noisy signal of resonator reflections (with slightly offset resonances and different line widths).

In [ ]:
# define frequency and complex signal
fvals = np.linspace(-5, 5, 101)

# signal: three different traces with different resonances and linewidths
svals_1 = (2j * fvals - 1.0) / (2j * fvals + 1.0)
svals_2 = (2j * (fvals-0.5) - 2.0) / (2j * (fvals-0.5) + 2.0)
svals_3 = (2j * (fvals+0.5) - 0.5) / (2j * (fvals+0.5) + 0.5)

# set up qcodes
frq = Parameter('detuning', unit='MHz')
sig1 = Parameter('reflection_1')
sig2 = Parameter('reflection_2')
sig3 = Parameter('reflection_3')

station = qc.Station(frq, sig)
exp = load_or_create_experiment('mock_resonator_sweep', sample_name='no sample')

# set up measurement
meas = Measurement(exp, station)
meas.register_parameter(frq, paramtype='array')
meas.register_custom_parameter('repetition')
meas.register_parameter(sig1, setpoints=('repetition', frq), paramtype='array')
meas.register_parameter(sig2, setpoints=('repetition', frq), paramtype='array')
meas.register_parameter(sig3, setpoints=('repetition', frq), paramtype='array')
meas.write_period = 2

# start measuring
with meas.run() as datasaver:
    for n in range(50):
        datasaver.add_result(
            (frq, fvals),
            ('repetition', n),
            (sig1, svals_1 + np.random.normal(size=fvals.size, scale=0.5) 
                + 1j*np.random.normal(size=fvals.size, scale=0.5)),
            (sig2, svals_2 + np.random.normal(size=fvals.size, scale=0.5)
                + 1j*np.random.normal(size=fvals.size, scale=0.5)),
            (sig3, svals_3 + np.random.normal(size=fvals.size, scale=0.5)
                + 1j*np.random.normal(size=fvals.size, scale=0.5)),
        )